In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

##### Парсинг сайта

In [3]:
from contextlib import closing
from selenium.webdriver import Safari # pip install selenium
from selenium.webdriver.support.ui import WebDriverWait

In [4]:
import sys
from selenium.webdriver.support.ui import Select

In [197]:
from selenium.webdriver import Chrome

chromedriver_loc = './chromedriver'
browser = Chrome(executable_path=chromedriver_loc)

# makes the browser wait if it can't find an element
browser.implicitly_wait(10)


In [198]:
browser.get("http://www.roszdravnadzor.ru/services/lssearch")


In [199]:
browser.find_element_by_link_text('Расширенный поиск').click()

In [200]:
status = browser.find_element_by_id('id_status')

In [201]:
select = Select(status) 

In [203]:
select.select_by_visible_text("контрафактное ЛС") 

In [204]:
browser.find_element_by_class_name('search-form-extended-submit').click()

In [205]:
from tqdm import tqdm
import time

In [206]:
columns=['ТН','Упаковка','Серия','Производитель', 'Страна','Статус ЛС','Тип','Масштаб','Информационное письмо']
med_df = pd.DataFrame(columns=columns)
for page in tqdm(range(1000)):
    content = browser.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(content,"html.parser")
    for i, data in enumerate(soup.find_all('tbody')[1]):
        if i != 0:
            row = []
            for col in range(8):
                row.append(data.find_all('td')[col].text)
            try:
                row.append(data.find_all('a')[0].get('href'))
            except:
                row.append('')
            med_df = pd.concat([med_df, pd.DataFrame([row], columns=columns)])
    success = False
    attempt = 0
    while not (success or attempt > 10):
        try:
            attempt += 1
            browser.find_element_by_link_text('Следующая').click()
            time.sleep(3)
            success = True
        except:
            print(attempt)
    if attempt > 10:
        break


  0%|          | 0/1000 [00:00<?, ?it/s]
Exception in thread Thread-13:
Traceback (most recent call last):
  File "//anaconda/envs/py36/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "//anaconda/envs/py36/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "//anaconda/envs/py36/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

  0%|          | 5/1000 [00:16<56:17,  3.39s/it]

1
2
3
4
5
6
7
8
9
10
11


In [208]:
med_df.drop_duplicates().to_csv('med_df_contraf.csv')

In [212]:
med_df = pd.read_csv('med_df_text.csv')

In [214]:
med_given = pd.read_excel('./Report_Database.xlsx', encoding="utf_8")

#### Составление словаря оценок

In [215]:
data_common = pd.merge(med_df, med_given, how='left', on='Серия')
data_common = data_common[~pd.isnull(data_common['Лаборатория'])]
data_common.reset_index(inplace=True)

In [269]:
# топовые причины
dict(
    zip(list(med_given['Причины забраковки'].value_counts().keys()[:25]), 
    list(med_given['Причины забраковки'].value_counts().values[:25]))
)

{'В связи с преостановлением реализации фармацевтической субстанции «Толперизона гидрохлорид», ': 58,
 'Выявление разночтений в документах, ': 29,
 'Выявлены несоответствия утвержденным нормам спецификации, в ходе изучения стабильности, ': 26,
 'Герметичность, ': 67,
 'Количественное определение, ': 164,
 'Количественное определение:Пропилпарагидроксибензоат, ': 37,
 'Лекарственный препарат изготовлен из фармацевтической субстанции «Орнидазол» запись о которой отсутствует в Государственном реестре лекарственных средств., ': 41,
 'Маркировка, ': 111,
 'Механические включения, ': 23,
 'Необходимость гармонизации регистрационного досье и нормативной документации., ': 100,
 'Необходимость гармонизации регистрационных документов., ': 212,
 'Обнаружение частиц полиуретана в упаковочном оборудовании., ': 38,
 'Остаточные органические растворители, ': 65,
 'Отзыв декларации о соответствии, ': 26,
 'Отзыв производителем, ': 23,
 'Отмена государственной регистрации., ': 49,
 'Отмена действия сер

In [433]:
# экспертная оценка

dict_reasons = {
'Упаковка': 1,
'инструкция на турецком языке': 1,

'Маркировка и инструкция': 2,
'Маркировка на ампулах стерта': 2,
'Маркировка': 2,
    
'Отзыв декларации о соответствии': 5,
'Отзыв производителем': 5,
'Отмена государственной регистрации': 5,
'Прекращение действия декларации о соответствии': 5,
'Отзыв производителем декларации о соответствии': 5,
'В связи с преостановлением реализации фармацевтической субстанции': 5,
'Необходимость гармонизации регистрационного досье и нормативной документации., ': 5,
'Необходимость гармонизации регистрационных документов., ': 5,
'Отмена действия сертификата соответствия': 5,
    
'Описание (': 6,
'Описание,': 6,
'Цветность': 6,
'запись о которой отсутствует в Государственном реестре лекарственных средств': 6,

'Количественное определение': 8,
'Количественное определение:Пропилпарагидроксибензоат': 8,
'Герметичность, ': 8,
'Растворение': 8,
    
'Микробиологическая чистота': 10,

'Фальсификация' : 10, 
'Подлинность' : 10, 
'Признаки фальсификации' : 10,
'Отмена действия сертификата соответствия' : 10,
'На этикетках флаконов отсутствуют серия и срок годности' : 10,
'Указание на первичной и вторичной упаковках: Дата изг Годен до' : 10, 
'Стабильность препарата' : 10,
'Развитие нежелательных реакций' : 10,
'Растворение' : 10, 
'Несоответствие в процессе производства' : 10,
'Выявление разночтений в документах': 10,
'Выявлены несоответствия утвержденным нормам спецификации, в ходе изучения стабильности': 10,
'Механические включения': 10,
'Обнаружение частиц полиуретана в упаковочном оборудовании': 10,
'Остаточные органические растворители': 10,

'Показатель в письме не указан': 1,
'Посторонние примеси': 10,
'Развитие нежелательных реакций': 10
 }

In [285]:
res =[]
for text in med_given['Причины забраковки']:
    if 'декларации' in str(text):
        res.append(text)


#### Если посмотреть на статус ЛС, можно вытащить все причины фальсифицикации

In [292]:
data_common['Причины забраковки_y'][data_common['Статус ЛС']=='фальсифицированное ЛС'].unique()

array(['Фальсификация, ', 'Подлинность, ', 'Признаки фальсификации, ',
       'Отмена действия сертификата соответствия, ',
       'На этикетках флаконов отсутствуют серия и срок годности, ',
       'Указание на первичной и вторичной упаковках: «Дата изг.: 11 2016», «Годен до: 09 2018»., ',
       'В связи с преостановлением реализации фармацевтической субстанции «Толперизона гидрохлорид», ',
       'Упаковка, Извлекаемый объем, ',
       'Техническая ошибка в маркировке номера серии на этикетке самоклеящейся на баллоне, ',
       'Показатель в письме не указан, ', 'Стабильность препарата., ',
       'Маркировка, ', 'Развитие нежелательных реакций, ',
       'Растворение, ', 'Несоответствие в процессе производства, ',
       'Однородность дозирования, Количественное определение, ',
       'Лекарственный препарат изготовлен из фармацевтической субстанции, реализация  которой  преостановлена, '],
      dtype=object)

#### Обрабатываем все собранные данные в одну таблицу med_final

In [337]:
med_unique = pd.read_csv('med_unique_docs.csv')

In [326]:
a = med_given[['Наименование товара', 'Производитель', 'Номер документа', 'Причины забраковки']]
med_given_d = a.drop_duplicates()

In [362]:
b = med_df[['ТН', 'Производитель', 'Информационное письмо', 'Приказ']][~pd.isnull(med_df['Приказ'])]
med_df_d = b.drop_duplicates()


In [352]:
med_unique_d = med_unique[['Информационное письмо', 'Причина']][~pd.isnull(med_unique['Причина'])]

In [365]:
med_given_d.head()

,Наименование товара,Производитель,Номер документа,Причины забраковки
0,"Хлоргексидин р-р д/местн. и наружн. прим. 0,05...",Кемеровская ФФ,01И-583_18,"Посторонние примеси, рН,"
1,Диара (лоперамид) таб. жев. 2мг N12,Оболенское ФП АО,01И-527_18,Прекращение действия декларации о соответствии...
4,"Кетопрофен гель 2,5% 40г",Софарма АО,01И-518_18,"Количественное определение. Бензалкония хлорид.,"
5,Контрактубекс гель 50г,Мерц Фарма ГмбХ и Ко.КГаА,01И-529_18,"Количественное определение. Аллантоин., Прекра..."
6,Метролакэр р-р д/инф. 5мг/мл 100мл N1,Ла Кэр Фарма Лимитед,01И-526_18,Прекращение действия декларации о соответствии...


In [363]:
med_df_d.head()

,ТН,Производитель,Информационное письмо,Приказ
3,5-НОК®,Лек д.д.,?downloadletter=21256,219957МинистерствоздравоохраненияРоссийскойФед...
15,Абакавир-АВС,"Общество с ограниченной ответственностью ""Техн...",?downloadletter=23684,Министерство здравоохранения Российской Федера...
16,Абакавир-АВС,"Общество с ограниченной ответственностью ""Техн...",?downloadletter=23655,Министерство здравоохранения Российской Федера...
21,Абактал®,Лек д.д.,?downloadletter=24685,Министерство здравоохранения Российской Федера...
23,Абактал®,Лек д.д.,?downloadletter=27619,Министерство здравоохранения Российской Федера...


In [364]:
med_unique_d.head()

,Информационное письмо,Причина
0,?downloadletter=21256,Несоответствие процессе производства
1,?downloadletter=23684,Маркировка
2,?downloadletter=23655,Маркировка
3,?downloadletter=24685,Прекращение действия декларации соответствии
4,?downloadletter=27619,Количественное определение


In [366]:
med_df_with_reason = pd.merge(med_df_d, med_unique_d, on = ['Информационное письмо'], how = 'left')
med_df_with_reason = med_df_with_reason[~pd.isnull(med_df_with_reason['Причина'])]

In [506]:
med_given_d.head()

,Наименование товара,Производитель,Номер документа,Причины забраковки
0,"Хлоргексидин р-р д/местн. и наружн. прим. 0,05...",Кемеровская ФФ,01И-583_18,"Посторонние примеси, рН,"
1,Диара (лоперамид) таб. жев. 2мг N12,Оболенское ФП АО,01И-527_18,Прекращение действия декларации о соответствии...
4,"Кетопрофен гель 2,5% 40г",Софарма АО,01И-518_18,"Количественное определение. Бензалкония хлорид.,"
5,Контрактубекс гель 50г,Мерц Фарма ГмбХ и Ко.КГаА,01И-529_18,"Количественное определение. Аллантоин., Прекра..."
6,Метролакэр р-р д/инф. 5мг/мл 100мл N1,Ла Кэр Фарма Лимитед,01И-526_18,Прекращение действия декларации о соответствии...


In [507]:
med_df_with_reason.head()

,ТН,Производитель,Информационное письмо,Приказ,Причина
0,5-НОК®,Лек д.д.,?downloadletter=21256,219957МинистерствоздравоохраненияРоссийскойФед...,Несоответствие процессе производства
1,Абакавир-АВС,"Общество с ограниченной ответственностью ""Техн...",?downloadletter=23684,Министерство здравоохранения Российской Федера...,Маркировка
2,Абакавир-АВС,"Общество с ограниченной ответственностью ""Техн...",?downloadletter=23655,Министерство здравоохранения Российской Федера...,Маркировка
3,Абактал®,Лек д.д.,?downloadletter=24685,Министерство здравоохранения Российской Федера...,Прекращение действия декларации соответствии
4,Абактал®,Лек д.д.,?downloadletter=27619,Министерство здравоохранения Российской Федера...,Количественное определение


In [395]:
med_prep = pd.read_excel('./grls2018-03-17-1.xls', encoding="utf_8", header=1)

In [410]:
med_names = med_prep[['Торговое наименование\nлекарственного препарата', 'Международное непатентованное или химическое наименование']].drop_duplicates().reset_index().drop('index', axis=1)
med_names.columns = ['ТН','МНН']

In [417]:
# САМАЯ ВАЖНАЯ ТАБЛИЦА
med_final = pd.merge(med_df_with_reason, med_names, on='ТН', how='left')

In [421]:
med_final[med_final['МНН'] == 'Дротаверин']

,ТН,Производитель,Информационное письмо,Приказ,Причина,МНН
1830,Дроверин,"Открытое акционерное общество ""Московское прои...",?downloadletter=27479,Министерство здравоохранения Российской Федера...,Стабильность препарата,Дротаверин
1831,Дроверин,"Открытое акционерное общество ""Московское прои...",?downloadletter=27153,Министерство здравоохранения Российской Федера...,Стабильность препарата,Дротаверин
1832,Дротаверин,"ЗАО ""Вифитех""",?downloadletter=24790,Министерство здравоохранения Российской Федера...,Цветность,Дротаверин
1833,Дротаверин,"ОАО ""Борисовский завод медицинских препаратов""",?downloadletter=25430,I/Il/Il I//Il I 2074459 Министерство здравоох...,Прекращение действия декларации соответствии,Дротаверин
1834,Дротаверин,Федеральное государственное унитарное предприя...,?downloadletter=27579,Министерство здравоохранения Российской Федера...,Механические включения,Дротаверин
1835,Дротаверин,Федеральное государственное унитарное предприя...,?downloadletter=27996,вL'Министерство здравоохранения Российской Фед...,Механические включения,Дротаверин
1836,Дротаверин,Федеральное государственное унитарное предприя...,?downloadletter=27429,Министерство здравоохранения Российской Федера...,Механические включения,Дротаверин
1837,Дротаверин,Федеральное государственное унитарное предприя...,?downloadletter=23207,Министерство здравоохранения Российской Федера...,Механические включения,Дротаверин
1838,Дротаверин,Дальхимфарм ОАО,?downloadletter=15395,Мииистерствоздравоохраиенияисоциальногоразвити...,Маркировка,Дротаверин
1839,Дротаверин,Дальхимфарм ОАО,?downloadletter=15555,Мииистерствоздравоохраненияисоциальиогоразвити...,"Описание,",Дротаверин


In [427]:
# Пример
# med_final.loc[1845, 'Приказ']

In [438]:
dict_reasons = {
'Упаковка': 1,
'инструкция турецком языке': 1,

'Маркировка инструкция': 2,
'Маркировка ампулах стерта': 2,
'Маркировка': 2,
    
'Отзыв декларации соответствии': 5,
'Отзыв производителем': 5,
'Отмена государственной регистрации': 5,
'Прекращение действия декларации соответствии': 5,
'Отзыв производителем декларации соответствии': 5,
'связи преостановлением реализации фармацевтической субстанции': 5,
'Необходимость гармонизации регистрационного досье нормативной документации': 5,
'Необходимость гармонизации регистрационных документов': 5,
'Отмена действия сертификата соответствия': 5,
    
'Описание (': 6,
'Описание,': 6,
'Цветность': 6,
'запись отсутствует Государственном реестре': 6,
'запись о которой отсутствует в Государственном реестре лекарственных средств': 6,

'Количественное определение': 8,
'Герметичность': 8,
'Растворение': 8,
    
'Микробиологическая чистота': 10,

'Фальсификация' : 10, 
'Подлинность' : 10, 
'Признаки фальсификации' : 10,
'Отмена действия сертификата соответствия' : 10,
'этикетках флаконов отсутствуют серия срок годности' : 10,
'Указание первичной вторичной упаковках' : 10, 
'Стабильность препарата' : 10,
'Развитие нежелательных реакций' : 10,
'Растворение' : 10, 
'Несоответствие процессе производства' : 10,
'Выявление разночтений документах': 10,
'Выявлены несоответствия утвержденным нормам спецификации изучения стабильности': 10,
'Механические включения': 10,
'Обнаружение частиц полиуретана упаковочном оборудовании': 10,
'Остаточные органические растворители': 10,

'Показатель письме указан': 1,
'Посторонние примеси': 10,
'Развитие нежелательных реакций': 10
 }

#### Создаем словари для быстрого доступа

In [465]:
dict_med_names = {}
for i in range(len(med_names)):
    if med_names.loc[i, 'ТН'] not  in dict_med_names:
        dict_med_names[med_names.loc[i, 'ТН']] = med_names.loc[i, 'МНН']
dict_tn_prod = {}
for i in range(len(med_final)):
    if med_final.loc[i, 'ТН'] not in dict_tn_prod:
        dict_tn_prod[med_final.loc[i, 'ТН']] = med_final.loc[i, 'Производитель']
        
dict_tn_prod_all = {}
for i in range(len(med_prep)):
    if med_prep.loc[i, 'Торговое наименование\nлекарственного препарата'] not in dict_tn_prod_all:
        dict_tn_prod_all[med_prep.loc[i, 'Торговое наименование\nлекарственного препарата']] = med_prep.loc[i, 'Юридическое лицо, на имя которого выдано регистрационное удостоверение']
        
dict_producers = {}
for i in range(len(med_final)):
    if med_final.loc[i, 'Производитель'] not in dict_producers:
        dict_producers[med_final.loc[i, 'Производитель']] = dict_reasons[med_final.loc[i, 'Причина']]
    else:
        dict_producers[med_final.loc[i, 'Производитель']] += dict_reasons[med_final.loc[i, 'Причина']]
        
dict_producers_cnt = {}
for i in range(len(med_final)):
    if med_final.loc[i, 'Производитель'] not in dict_producers_cnt:
        dict_producers_cnt[med_final.loc[i, 'Производитель']] = 1
    else:
        dict_producers_cnt[med_final.loc[i, 'Производитель']] += 1

#### Сохранение таблиц

In [502]:
med_final.to_csv('med_final.csv')
med_final = pd.read_csv('med_final.csv', index_col=0)

In [504]:
med_names.to_csv('med_names.csv')
med_names = pd.read_csv('med_names.csv', index_col=0)

In [505]:
med_prep.to_csv('med_prep.csv')
med_prep = pd.read_csv('med_prep.csv', index_col=0)

In [444]:
med_final[med_final['МНН'] == 'Дротаверин']['Производитель'].unique()

array(['Открытое акционерное общество "Московское произ...',
       'ЗАО "Вифитех"', 'ОАО "Борисовский завод медицинских препаратов"',
       'Федеральное государственное унитарное предприят...',
       'Дальхимфарм ОАО',
       'Федеральное казенное предприятие "Армавирская б...',
       'Вифитех ЗАО', 'Биохимик ОАО',
       'Федеральное казённое предприятие "Армавирская б...',
       'Компания Деко ООО', 'Ирбитский ХФЗ ОАО', 'Фармпроект ЗАО',
       'ОАО "Ирбитский химико-фармацевтический завод"',
       'ЗАО "Фармпроект"', 'ООО МЦ "Эллара"'], dtype=object)

#### Функция вывода рейтинга по МНН

In [486]:
def sorted_producers(medicine_name):
    medicine_names = list(med_names[med_names['МНН'] == medicine_name]['ТН'].unique())
    medicine_names2 = list(med_final[med_names['МНН'] == medicine_name]['ТН'].unique())
    medicine_names += medicine_names2
    rating = []
    for name in medicine_names:
        if name in dict_tn_prod:
            prod = dict_tn_prod[name]
            if prod in dict_producers:
                rating.append((name, prod, dict_producers[prod], dict_producers_cnt[prod]))
        elif name in dict_tn_prod_all:
            prod = dict_tn_prod_all[name]
            rating.append((name, prod, 0, 0))
        else:
            rating.append((name, 1, 1, 1))
    rating = sorted(rating, key=lambda x: x[2])
    return rating

In [487]:
sorted_producers('Дротаверин')

//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


[('Дротаверина гидрохлорид',
  'ОАО "Усолье-Сибирский химико-фармацевтический завод"',
  0,
  0),
 ('Дротаверин-OBL',
  'Закрытое акционерное общество «Фармацевтическое предприятие «Оболенское» (ЗАО «ФП «Оболенское»)',
  0,
  0),
 ('Пле-Спа', 'Плетхико Фармасьютикалз Лтд', 0, 0),
 ('Дротаверин-Тева', 'Тева Фармацевтические Предприятия Лтд', 0, 0),
 ('Дротаверин-СОЛОфарм',
  'Общество с ограниченной ответственностью "Гротекс" (ООО "Гротекс")',
  0,
  0),
 ('Биошпа', 'Биофарма ЗАО', 0, 0),
 ('Дротаверин-Эллара', 'Эллара МЦ ООО', 0, 0),
 ('Спазмол®',
  'Открытое акционерное общество "Фармстандарт-Уфимский витаминный завод" (ОАО "Фармстандарт-УфаВИТА")',
  0,
  0),
 ('Спазмонет форте',
  'Общество с ограниченной ответственностью "Крка-Рус"',
  0,
  0),
 ('Спазмонет® форте',
  'Общество с ограниченной ответственностью "КРКА-РУС" (ООО "КРКА-РУС")',
  0,
  0),
 ('Спазмонет', 'Общество с ограниченной ответственностью "Крка-Рус"', 0, 0),
 ('Спазмонет®',
  'Общество с ограниченной ответственност

#### Функция вывода общего рейтинга

In [508]:
def all_rating():
    rating = []
    for name in dict_producers.keys():
        if name in dict_producers_cnt:
            rating.append((name, np.round(dict_producers[name]/dict_producers_cnt[name], 2)))
    rating = sorted(rating, key=lambda x: x[1])
    return rating

In [509]:
all_rating()

[('ООО "Мосфарм"', 1.0),
 ('Пик-Фарма ПРО ООО', 1.0),
 ('Аптека ФГБУ "Медицинский радиологический научны...', 1.0),
 ('МУП г. Владивостока "Аптека № 219",  690109, Пр...', 1.0),
 ('ООО Аромасинтез ', 1.0),
 ('Диафарм институт молекулярной диагностики ЗАО', 1.0),
 ('Шаньдун Шэнлу Фармасьютикал Ко.Лтд', 1.0),
 ('Борисовский завод медицинских препаратов ОАО, (...', 1.0),
 ('Грин Кросс Корпорейшен', 1.5),
 ('Химфарм АО', 1.75),
 ('Общество с ограниченной ответственностью "Техно...', 2.0),
 ('Реплекфарм АО', 2.0),
 ('Кедрион С.п.А.', 2.0),
 ('Лилли Франс С.а.С.', 2.0),
 ('Фарос-21 ООО', 2.0),
 ('Фармтехнология ООО', 2.0),
 ('Митим С.р.Л.', 2.0),
 ('ООО "Бион"', 2.0),
 ('Ваке спол.с.р.о.', 2.0),
 ('Югоремедия Фабрика лекарств АО', 2.0),
 ('Секонд Фарма Ко.Лтд', 2.0),
 ('Общество с ограниченной отвественностью "Гиппок...', 2.0),
 ('Бен Венью Лабораторис Инк', 2.0),
 ('ЗАО "Брынцалов А"', 2.0),
 ('СОАО "Ферейн"', 2.0),
 ('Шандонг Вейфанг Фармасьютикал Фэктори Ко.Лтд', 2.0),
 ('Биокад ЗАО', 2.0

In [420]:
med_final['МНН'].value_counts()

~                                                                                                    1387
-                                                                                                     192
Ацетилсалициловая кислота                                                                             125
Аммиак                                                                                                123
Водорода пероксид                                                                                     102
Хлоргексидин                                                                                           89
Активированный уголь                                                                                   89
Парацетамол                                                                                            77
Дексаметазон                                                                                           76
Кальция глюконат                              